## Install libraries and set up keys

In [1]:
%%capture

!pip install llama-index                         # llama-index core
!pip install llama-index-llms-openai           # OpenAI bindings
!pip install llama-index-llms-anthropic          # Anthropic bindings
!pip install llama-index-embeddings-huggingface  # HuggingFace Embeddings

!pip install google-search-results  # Serp API
!pip install pypdf                  # Parse PDF docs
!pip install sentence_transformers  # Embedding
!pip install chromadb               # Vector DB

In [7]:
import os
from google.colab import userdata

os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')
os.environ['SERPAPI_API_KEY'] = userdata.get('SERPAPI_API_KEY')
os.environ['ANTHROPIC_API_KEY'] = userdata.get('ANTHROPIC_API_KEY')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [3]:
!rm -rf ./repo
!git clone --depth 1 https://github.com/adeshmukh/gaiip-intro-orchestration.git ./repo

Cloning into './repo'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 15 (delta 1), reused 12 (delta 1), pack-reused 0
Receiving objects: 100% (15/15), 632.62 KiB | 2.01 MiB/s, done.
Resolving deltas: 100% (1/1), done.


## Load PDFs in Vector DB

In [4]:
%%capture

from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.tools import QueryEngineTool, ToolMetadata

# Ignore pypdf warnings
import logging
logging.getLogger("pypdf").setLevel(logging.ERROR)

# load data
base_path = "./repo/pdfs"
pdf_files = os.listdir(base_path)
pdf_file_paths = [os.path.join(base_path, file) for file in pdf_files]

docs = SimpleDirectoryReader(
    input_files=pdf_file_paths
).load_data()

# build index
from llama_index.core import Settings

# embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
Settings.embed_model = embed_model
pdf_index = VectorStoreIndex.from_documents(docs, show_progress=True)

# persist index
pdf_index.storage_context.persist(persist_dir="./storage/pdfs")


In [8]:
pdf_engine = pdf_index.as_query_engine(similarity_top_k=3)
vector_db_tool = QueryEngineTool(
        query_engine=pdf_engine,
        metadata=ToolMetadata(
            name="us_national_budget_fy24",
            description=(
                "Provides information about US national budget for the fiscal year 2024. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    )

## Zero-shot ReAct agent

In [17]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.llms.anthropic import Anthropic
from llama_index.core.tools import BaseTool, FunctionTool

In [13]:

def multiply(a: float, b: float) -> int:
    """Multiply two numbers and returns the result number"""
    return a * b

def add(a: float, b: float) -> int:
    """Add two numbers and returns the result number"""
    return a + b

In [18]:
multiply_tool = FunctionTool.from_defaults(fn=multiply)
add_tool = FunctionTool.from_defaults(fn=add)

# llm = OpenAI(model="gpt-3.5-turbo-instruct")
llm = Anthropic(model='claude-3-opus-20240229')
agent = ReActAgent.from_tools([multiply_tool, add_tool, vector_db_tool], llm=llm, verbose=True)

In [19]:
response = agent.chat("What is the total requested budget for NASA and NSF for Fiscal Year 2024?")

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: us_national_budget_fy24
Action Input: {'input': 'What is the total requested budget for NASA and NSF for Fiscal Year 2024?'}
Observation: The total requested budget for NASA for Fiscal Year 2024 is $27.2 billion, while the total requested budget for NSF for Fiscal Year 2024 is $11.3 billion.
Thought: I have the NASA and NSF budget numbers, but to answer the question I need to add them together to get the total.
Action: add
Action Input: {'a': 27.2, 'b': 11.3}
Observation: 38.5
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The total requested budget for NASA and NSF combined for Fiscal Year 2024 is $38.5 billion. NASA's requested budget is $27.2 billion and NSF's requested budget is $11.3 billion, which adds up to a total of $38.5 billion.
